# Analysisng data in Edmonton, Alberta
### Armin Norouzi
### MARCH 26, 2021



### improting Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder

import folium

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

from pandas.io.json import json_normalize 

### Scraping the data form Wikipedia
#####  Processed data is availible in [here](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T) 

In [2]:
df = pd.read_csv('AlbertaDb.csv')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.03646,-110.67925
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.04968,-113.96432
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12606,-114.143158
3,T4A,Airdrie,East Airdrie,51.27245,-113.98698
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


### Removing "Not assigned" data

In [3]:
df = df[df['Borough'] != "Not assigned"]
df = df[df['Neighborhood'] != "Not assigned"]
df = df[df['Latitude'] != "Not assigned"]
df = df[df['Longitude'] != "Not assigned"]
df = df[df['Postal Code'] != "Not assigned"]

df = df.reset_index(drop=True)

In [4]:
latitude = df['Latitude'][0]
longitude = df['Longitude'][0]
map_AB = folium.Map(location=[latitude,longitude],zoom_start=5)

# add markers to map
for lat,lng,label in zip(df['Latitude'],df['Longitude'],df['Neighborhood']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_AB)  
    
map_AB

### Edmonton Data

In [5]:
df_EDM = df[df['Borough'] == "Edmonton"]
df_EDM = df_EDM[df_EDM['Postal Code'] != "T6S"]
df_EDM = df_EDM.reset_index(drop=True)

df_EDM.head(50)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
1,T6A,Edmonton,North Capilano,53.5483,-113.408
2,T5B,Edmonton,"East North Central, West Beverly",53.5766,-113.4608
3,T6B,Edmonton,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404
4,T5C,Edmonton,Central Londonderry,53.6129,-113.4572
5,T6C,Edmonton,Central Bonnie Doon,53.5182,-113.4769
6,T5E,Edmonton,"West Londonderry, East Calder",53.5923,-113.5168
7,T6E,Edmonton,"South Bonnie Doon, East University",53.5087,-113.5078
8,T5G,Edmonton,"North Central, Queen Mary Park, Blatchford",53.5682,-113.4822
9,T6G,Edmonton,"West University, Strathcona Place",53.5248,-113.5334


### Visualizing Edmonton Neiberhods

In [6]:
latitude = df_EDM['Latitude'][0]
longitude = df_EDM['Longitude'][0]
map_EDM = folium.Map(location=[latitude,longitude],zoom_start=10)

# add markers to map
for lat,lng,label in zip(df_EDM['Latitude'],df_EDM['Longitude'],df_EDM['Neighborhood']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_EDM)  
    
map_EDM

## Foursquare API and function to analyze Edmonton neighborhoods

In [7]:
CLIENT_ID = 'SP15RBBMRAQV4VNI3M4PHBZRETJTIOCHSRIUGCGT512KASIE' # my Foursquare ID
CLIENT_SECRET = 'AWGZBLUTRZFGTWSRHUGT0NUHYESBCXDHP1NH4W05PTES1DAF' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 700
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: SP15RBBMRAQV4VNI3M4PHBZRETJTIOCHSRIUGCGT512KASIE
CLIENT_SECRET: AWGZBLUTRZFGTWSRHUGT0NUHYESBCXDHP1NH4W05PTES1DAF


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
Edm_venues = getNearbyVenues(names=df_EDM['Neighborhood'],
                                   latitudes=df_EDM['Latitude'],
                                   longitudes=df_EDM['Longitude']
                                  )



West Clareview, East Londonderry
North Capilano
East North Central, West Beverly
SE Capilano, West Southeast Industrial, East Bonnie Doon
Central Londonderry
Central Bonnie Doon
West Londonderry, East Calder
South Bonnie Doon, East University
North Central, Queen Mary Park, Blatchford
West University, Strathcona Place
NorthDowntown Fringe, East Downtown Fringe
Southgate, North Riverbend
North Downtown
Kaskitayo, Aspen Gardens
South Downtown, South Downtown Fringe (Alberta Provincial Government)
West Mill Woods
North Westmount, West Calder, East Mistatim
East Mill Woods
South Westmount, Groat Estate, East Northwest Industrial
Southwest Edmonton
Glenora, SW Downtown Fringe
South Industrial
North Jasper Place
East Southeast Industrial, South Clover Bar
Central Jasper Place, Buena Vista
Southgate, North Riverbend
West Northwest Industrial, Winterburn
West Jasper Place, West Edmonton Mall
The Meadows
Central Mistatim
The Palisades, West Castle Downs
Central Beverly
Heritage Valley
East Cast

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"West Clareview, East Londonderry",53.5899,-113.4413,7-Eleven,53.584696,-113.444569,Convenience Store
1,"West Clareview, East Londonderry",53.5899,-113.4413,Buffet Royale Carvery,53.587229,-113.439075,Buffet
2,"West Clareview, East Londonderry",53.5899,-113.4413,Café del Sol,53.592441,-113.441455,Mexican Restaurant
3,"West Clareview, East Londonderry",53.5899,-113.4413,Red Claw Gaming,53.586937,-113.439775,Toy / Game Store
4,"West Clareview, East Londonderry",53.5899,-113.4413,My Grandma's Attic,53.586033,-113.441629,Record Shop
...,...,...,...,...,...,...,...
551,West Lake District,53.5966,-113.4882,Humpty's Family Restaurant,53.592879,-113.492299,American Restaurant
552,West Lake District,53.5966,-113.4882,Gold's Gym,53.601164,-113.489413,Gym
553,West Lake District,53.5966,-113.4882,Carl’s Jr.,53.601249,-113.493042,Burger Joint
554,West Lake District,53.5966,-113.4882,Fairfield Inn & Suites by Marriott Edmonton North,53.601756,-113.492771,Hotel


In [26]:
Edm_venues.groupby('Neighborhood').count().head(40)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Beverly,13,13,13,13,13,13
Central Bonnie Doon,23,23,23,23,23,23
"Central Jasper Place, Buena Vista",10,10,10,10,10,10
Central Londonderry,4,4,4,4,4,4
Central Mistatim,4,4,4,4,4,4
East Castledowns,6,6,6,6,6,6
East Mill Woods,2,2,2,2,2,2
"East North Central, West Beverly",13,13,13,13,13,13
"East Southeast Industrial, South Clover Bar",5,5,5,5,5,5


## Analyzing top Neibhouds

In [10]:
latitude = df_EDM['Latitude'][12]
longitude = df_EDM['Longitude'][12]

radius = 700 
LIMIT = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
   longitude, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [11]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

D:\Users\norouziy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.location.crossStreet', 'venue.venuePage.id',
       'venue.location.neighborhood'],
      dtype='object')

### Visualizing Edmonton Neiberhods with nearby venues

In [12]:
latitude = df_EDM['Latitude'][0]
longitude = df_EDM['Longitude'][0]
map_EDM = folium.Map(location=[latitude,longitude],zoom_start=10)

# add markers to map
for lat,lng,label in zip(df_EDM['Latitude'],df_EDM['Longitude'],df_EDM['Neighborhood']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_EDM)  
    


# add markers to map
for lat,lng,label in zip(nearby_venues['venue.location.lat'],nearby_venues['venue.location.lng'],nearby_venues['venue.name']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_EDM)  
    
map_EDM

### Sumrizing the venus

In [13]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Remedy Cafe,Café,53.540783,-113.497405
1,The Old Spaghetti Factory,Italian Restaurant,53.543555,-113.497549
2,Corso 32,Italian Restaurant,53.540949,-113.498752
3,Sabor Divino,Restaurant,53.543152,-113.497528
4,The Cask & Barrel,Whisky Bar,53.540553,-113.499328


### Top venues

In [14]:
a=pd.Series(nearby_venues.categories)
a.value_counts()[:20]

Coffee Shop                9
Sandwich Place             6
Italian Restaurant         5
Pub                        4
Café                       4
Hotel                      4
Restaurant                 3
French Restaurant          3
Pizza Place                2
New American Restaurant    2
Rock Club                  2
Brewery                    2
Gym                        2
Nightclub                  2
Smoothie Shop              2
Asian Restaurant           2
Vietnamese Restaurant      2
Bar                        2
Mexican Restaurant         2
Steakhouse                 2
Name: categories, dtype: int64

## Most Common venues near neighborhood

#### One Hot Encoding of Features

In [15]:
Edm_venues = getNearbyVenues(names=df_EDM['Neighborhood'],
                                   latitudes=df_EDM['Latitude'],
                                   longitudes=df_EDM['Longitude']
                                  )

Edm_venues.groupby('Neighborhood').head(100)

West Clareview, East Londonderry
North Capilano
East North Central, West Beverly
SE Capilano, West Southeast Industrial, East Bonnie Doon
Central Londonderry
Central Bonnie Doon
West Londonderry, East Calder
South Bonnie Doon, East University
North Central, Queen Mary Park, Blatchford
West University, Strathcona Place
NorthDowntown Fringe, East Downtown Fringe
Southgate, North Riverbend
North Downtown
Kaskitayo, Aspen Gardens
South Downtown, South Downtown Fringe (Alberta Provincial Government)
West Mill Woods
North Westmount, West Calder, East Mistatim
East Mill Woods
South Westmount, Groat Estate, East Northwest Industrial
Southwest Edmonton
Glenora, SW Downtown Fringe
South Industrial
North Jasper Place
East Southeast Industrial, South Clover Bar
Central Jasper Place, Buena Vista
Southgate, North Riverbend
West Northwest Industrial, Winterburn
West Jasper Place, West Edmonton Mall
The Meadows
Central Mistatim
The Palisades, West Castle Downs
Central Beverly
Heritage Valley
East Cast

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"West Clareview, East Londonderry",53.5899,-113.4413,7-Eleven,53.584696,-113.444569,Convenience Store
1,"West Clareview, East Londonderry",53.5899,-113.4413,Buffet Royale Carvery,53.587229,-113.439075,Buffet
2,"West Clareview, East Londonderry",53.5899,-113.4413,Café del Sol,53.592441,-113.441455,Mexican Restaurant
3,"West Clareview, East Londonderry",53.5899,-113.4413,Red Claw Gaming,53.586937,-113.439775,Toy / Game Store
4,"West Clareview, East Londonderry",53.5899,-113.4413,My Grandma's Attic,53.586033,-113.441629,Record Shop
...,...,...,...,...,...,...,...
551,West Lake District,53.5966,-113.4882,Humpty's Family Restaurant,53.592879,-113.492299,American Restaurant
552,West Lake District,53.5966,-113.4882,Gold's Gym,53.601164,-113.489413,Gym
553,West Lake District,53.5966,-113.4882,Carl’s Jr.,53.601249,-113.493042,Burger Joint
554,West Lake District,53.5966,-113.4882,Fairfield Inn & Suites by Marriott Edmonton North,53.601756,-113.492771,Hotel


In [16]:
# one hot encoding
Edm_onehot = pd.get_dummies(Edm_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Edm_onehot['Neighborhood'] = Edm_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Edm_onehot.columns[-1]] + list(Edm_onehot.columns[:-1])
Edm_onehot = Edm_onehot[fixed_columns]
Edm_grouped = Edm_onehot.groupby('Neighborhood').mean().reset_index()
Edm_onehot.head(60)

,Neighborhood,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bakery,Bank,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Shop
0,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"West Clareview, East Londonderry",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
num_top_venues = 5
for hood in Edm_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp =Edm_grouped[Edm_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Central Beverly ----
             venue  freq
0      Pizza Place  0.15
1  Motorcycle Shop  0.08
2    Hot Dog Joint  0.08
3      Bus Station  0.08
4   Sandwich Place  0.08


---- Central Bonnie Doon ----
                 venue  freq
0    Convenience Store  0.09
1                Trail  0.09
2  American Restaurant  0.04
3         Hockey Arena  0.04
4           Comic Shop  0.04


---- Central Jasper Place, Buena Vista ----
                  venue  freq
0        Sandwich Place   0.1
1      Sushi Restaurant   0.1
2  Fast Food Restaurant   0.1
3           Salad Place   0.1
4                Bakery   0.1


---- Central Londonderry ----
               venue  freq
0     Ice Cream Shop  0.25
1   Department Store  0.25
2  Convenience Store  0.25
3        Pizza Place  0.25
4               Park  0.00


---- Central Mistatim ----
             venue  freq
0              Inn  0.25
1           Casino  0.25
2  Warehouse Store  0.25
3     Liquor Store  0.25
4           Office  0.00


---- East Castled

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Edm_grouped['Neighborhood']

for ind in np.arange(Edm_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Edm_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(40)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Beverly,Pizza Place,Hotel,Sandwich Place,Chinese Restaurant,Hot Dog Joint,Athletics & Sports,Bus Station,Furniture / Home Store,Motorcycle Shop,Liquor Store
1,Central Bonnie Doon,Convenience Store,Trail,Ice Cream Shop,Event Space,Music Venue,Market,Hockey Arena,Grocery Store,Gastropub,Ethiopian Restaurant
2,"Central Jasper Place, Buena Vista",Salad Place,Bakery,Pizza Place,Sushi Restaurant,Liquor Store,Fast Food Restaurant,Sandwich Place,Café,Convenience Store,Park
3,Central Londonderry,Ice Cream Shop,Convenience Store,Pizza Place,Department Store,Athletics & Sports,Filipino Restaurant,French Restaurant,Football Stadium,Food Truck,Food Court
4,Central Mistatim,Warehouse Store,Inn,Casino,Liquor Store,Wine Shop,Filipino Restaurant,Football Stadium,Food Truck,Food Court,Food & Drink Shop
5,East Castledowns,Bar,Pizza Place,Plaza,Convenience Store,Skating Rink,Karaoke Bar,Event Space,Food Court,Food & Drink Shop,Flower Shop
6,East Mill Woods,Pub,Bakery,Fast Food Restaurant,French Restaurant,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Filipino Restaurant
7,"East North Central, West Beverly",Pizza Place,Hotel,Sandwich Place,Chinese Restaurant,Hot Dog Joint,Athletics & Sports,Bus Station,Furniture / Home Store,Motorcycle Shop,Liquor Store
8,"East Southeast Industrial, South Clover Bar",Housing Development,Auto Garage,Shopping Mall,Construction & Landscaping,Baseball Field,Wine Shop,Filipino Restaurant,Football Stadium,Food Truck,Food Court
9,Ellerslie,Motorcycle Shop,Rental Car Location,Gymnastics Gym,Event Space,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Filipino Restaurant,Fast Food Restaurant


### K-Means Clustering Approach

In [20]:
# Using K-Means to cluster neighborhood into 3 clusters
Edm_grouped_clustering = Edm_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=3, random_state=0).fit(Edm_grouped_clustering)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [21]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Edm_merged =df_EDM.iloc[:37,:]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Edm_merged = Edm_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Edm_merged.head(50)# check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413,0,Bus Station,Convenience Store,Casino,Buffet,Light Rail Station,Comedy Club,Toy / Game Store,Record Shop,Hotel,Mexican Restaurant
1,T6A,Edmonton,North Capilano,53.5483,-113.408,0,Playground,Baseball Field,Ski Trail,Construction & Landscaping,Bus Station,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Flower Shop
2,T5B,Edmonton,"East North Central, West Beverly",53.5766,-113.4608,0,Pizza Place,Hotel,Sandwich Place,Chinese Restaurant,Hot Dog Joint,Athletics & Sports,Bus Station,Furniture / Home Store,Motorcycle Shop,Liquor Store
3,T6B,Edmonton,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404,0,Coffee Shop,Playground,Home Service,Bakery,Greek Restaurant,Wine Shop,Food Truck,Food Court,Food & Drink Shop,Flower Shop
4,T5C,Edmonton,Central Londonderry,53.6129,-113.4572,0,Ice Cream Shop,Convenience Store,Pizza Place,Department Store,Athletics & Sports,Filipino Restaurant,French Restaurant,Football Stadium,Food Truck,Food Court
5,T6C,Edmonton,Central Bonnie Doon,53.5182,-113.4769,0,Convenience Store,Trail,Ice Cream Shop,Event Space,Music Venue,Market,Hockey Arena,Grocery Store,Gastropub,Ethiopian Restaurant
6,T5E,Edmonton,"West Londonderry, East Calder",53.5923,-113.5168,0,Dog Run,Hockey Arena,Grocery Store,Comic Shop,Bakery,Baseball Field,Recreation Center,Arts & Crafts Store,Butcher,Ethiopian Restaurant
7,T6E,Edmonton,"South Bonnie Doon, East University",53.5087,-113.5078,0,American Restaurant,Coffee Shop,Mediterranean Restaurant,Pharmacy,Flower Shop,Chinese Restaurant,Food Truck,Food Court,Food & Drink Shop,Filipino Restaurant
8,T5G,Edmonton,"North Central, Queen Mary Park, Blatchford",53.5682,-113.4822,0,Bakery,Hot Dog Joint,Theater,Pharmacy,Restaurant,Café,Gas Station,Latin American Restaurant,Grocery Store,Pizza Place
9,T6G,Edmonton,"West University, Strathcona Place",53.5248,-113.5334,0,Coffee Shop,Pub,College Gym,Theater,Juice Bar,Donut Shop,Paper / Office Supplies Store,College Residence Hall,Sandwich Place,Fast Food Restaurant


In [23]:
kclusters = 10

In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map

markers_colors = []
for lat, lon, nei , cluster in zip(Edm_merged['Latitude'], 
                                   Edm_merged['Longitude'], 
                                   Edm_merged['Neighborhood'], 
                                   Edm_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']
